In [ ]:
import re
import nltk
import numpy as np
import pandas as pd
import re
from sklearn import feature_extraction
import matplotlib as mpl
import joblib
import os

In [ ]:
inputfile = "gittables"
headerinfos =[]
lengthinfo = []
filder_files = os.listdir(inputfile)
for filename in filder_files:
    files = os.listdir(inputfile +'\\'+str(filename))
    for file in files:
        temp = []
        check = True
        no_lines = 0
        with open(inputfile + '\\'+str(filename)+'\\'+str(file), "r", encoding='utf8') as f:
            for line in f:
                if check:
                    if not re.search(['[a-zA-Z]'], line):
                        check = False
                    else:
                        headerinfos.append(line)
                        check = False
                if not check:
                    no_lines += 1
        lengthinfo.append(no_lines)
joblib.dump(headerinfos, 'headerinfos.pkl')

In [ ]:
def splitting(headerline):
    if "\"" in headerline and sum(map(lambda x : 1 if '\"' in x else 0, headerline)) % 2 == 0:
        reststring=""
        escape=""
        positions = []
        count = 0
        for c in headerline:
            if c == "\"":
                positions.append(count)
                count+=1
            if len(positions) % 2 == 0 and c != "\"":
                reststring += c
            else:
                escape += c
        return splitting(reststring) + escape.split("\"")
    elif headerline[0] == ",":
        if ";" in headerline:
            temp = headerline.split(",")
            return [item for sublist in [x.split(";") for x in temp] for item in sublist]
        return headerline.split(",")
    elif ";" in headerline:
        return headerline.split(";")
    else:
        return headerline.split(";")

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def tokenstemmanizer(text):
    tokens = [word for w in nltk.sent_tokenize(text) for word in nltk.word_tokenize(w)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
def tokeizer(text):
    tokens = [word.lower() for w in nltk.sent_tokenize(text) for word in nltk.word_tokenize(w)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

stemmed = []
tokenized = []
for head in headerinfos:
    words_stemmed = tokenstemmanizer(head) 
    stemmed.extend(words_stemmed)
    words_tokenized = tokeizer(head)
    tokenized.extend(words_tokenized)

In [ ]:
import math
def dist_cosine_sim(vx,vy):
    a = 0
    b = 0
    c = 0
    for x in vx:
        for y in vy:
            a += x*y
            b += x*x
            c += y*y
    if (math.sqrt(b)*math.sqrt(c))==0:
        return 0
    return 1- (a / (math.sqrt(b)*math.sqrt(c)))

#~62000 Stunden
# mit cosine_similiarity: ~22386 Stunden ^= 2,5 Jahre


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenstemmanizer, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(headerinfos) # scipy.sparse._csr.csr_matrix
terms = tfidf_vectorizer.get_feature_names_out()

#joblib.dump(terms, 'tfidf_terms.pkl')

In [ ]:
tfidf_matrix = joblib.load('matrix.pkl')

In [ ]:
dist = []
k = tfidf_matrix.toarray()
len_k = len(k)

In [52]:
#terms = joblib.load('tfidf_terms.pkl')


In [53]:
print(terms)

['author' 'author titl' 'author titl url' 'comment' 'comment author'
 'comment author titl' 'content' 'content creat' 'creat' 'id' 'id type'
 'id type stori' 'parent' 'parent point' 'parent point comment' 'point'
 'point comment' 'point comment author' 'stori' 'stori parent'
 'stori parent point' 'titl' 'titl url' 'titl url content' 'type'
 'type stori' 'type stori parent' 'url' 'url content' 'url content creat']


In [ ]:
#from joblib import Memory
## https://medium.com/dataturtles/disk-caching-using-joblib-51372056afac

#cache_dir = "C:\\Users\\hanna\\Documents\\temp\\Sem10\\lakehouse\\cachedir"
#mem = Memory(cache_dir)
#func_mem = mem.cache(cosine_similarity, verbose = 0)
#dist_mem = func_mem(tfidf_unique)


In [ ]:

from sklearn.cluster import KMeans
num_clusters = 30
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [ ]:
import joblib
#joblib.dump(km, 'doc_cluster.pkl')

In [60]:
import joblib
km = joblib.load('doc_cluster.pkl')
#num_clusters = 30
#headerinfos = joblib.load('headerinfos.pkl')
tfidf_matrix = joblib.load('matrix.pkl')
#dist_allclusters = joblib.load('dist_allclusters.pkl')
#pos = joblib.load("pos.pkl")
#color = joblib.load("color.pkl")
num_clusters=30

In [61]:
clusters = km.labels_.tolist()

In [62]:
clusterindexes = []
for i in range(num_clusters):
    clusterindexes.append([])
for i in range(len(clusters)):
    clusterindexes[clusters[i]].append(i)

In [ ]:
headerinfos_by_cluster = []
for i_cluster in clusterindexes:
    temp = []
    for index in i_cluster:
        temp.append(headerinfos[index])
    headerinfos_by_cluster.append(temp)

In [63]:
tfidf_matrix_by_cluster = []
k = tfidf_matrix.toarray()
for cl_i in clusterindexes:
    temp = []
    for i in cl_i:
        temp.append(k[i])
    tfidf_matrix_by_cluster.append(temp)


In [65]:
count=0
for arr_cl in tfidf_matrix_by_cluster:
    if(len(arr_cl)) > 51000 or (len(arr_cl)==0):
        print(count)
    count+=1

0
1
2
3


In [ ]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
dist_allclusters = []

for arr_cl in tfidf_matrix_by_cluster:
    if(len(arr_cl)) > 51000 or (len(arr_cl)==0):
        continue
    dist_allclusters.append(1-cosine_similarity(csr_matrix(arr_cl)))
#joblib.dump(dist_allclusters, 'dist_allclusters.pkl')
#TODO

In [ ]:
#joblib.dump(dist_allclusters, 'dist_allclusters.pkl')

In [ ]:
print(terms)

In [ ]:
import numpy as np
dist_unique  = []
for c in dist_allclusters:
    if c not in dist_unique:
        dist_unique.append(c)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS
from scipy.sparse import csr_matrix

MDS()
pos = []
for c in dist_unique:
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
    pos.append(mds.fit_transform(c))
#joblib.dump(pos, 'pos.pkl')
#995m47.7s

In [ ]:
def clustermean (clusterarray):
    result = 0.0
    for elem in clusterarray:
        result += elem
    return result/len(clusterarray)

In [ ]:
import numpy as np
meanclust_dist = []
for clust in dist_allclusters:
    if len(clust) == 0:
        continue
    meanclust_dist.append(np.sum(clust)/(np.sum([len(cl) for cl in clust])))

In [ ]:
import joblib
pos_mean = joblib.load("pos_mean.pkl")

In [ ]:
#joblib.dump(pos, 'pos.pkl')

In [ ]:

#joblib.dump(pos_mean, 'pos_mean.pkl')

In [ ]:
import random
color = {}
r = lambda: random.randint(0,255)
for i in range(len(meanclust_dist)):
    color[i]='#{:02x}{:02x}{:02x}'.format(r(), r(), r())
print(color)

In [ ]:
#joblib.dump(color, 'color.pkl')

In [39]:
import pandas as pd
import matplotlib.pyplot as plt

count = 0
all_x = []
all_y = []
all_labels =[]
all_titles = []
for cluster in pos:
    xs, ys = cluster[:,0], cluster[:, 1]
    all_x.append([(x) for x in xs])
    all_y.append([(y) for y in ys])
    for i in range(len(xs)):
        all_labels.append(count)
        all_titles.append(str(count))
    count+=1

all_xs = [item for sublist in all_x for item in sublist]
all_ys = [item for sublist in all_y for item in sublist]

In [ ]:
all_ys = meanclust_dist
all_xs = range(len(meanclust_dist))
all_labels = all_xs
all_titles = [str(c) for c in all_xs]

In [ ]:

import pandas as pd
%matplotlib inline 
df = pd.DataFrame(dict(x=all_xs, y=all_ys, label=all_labels, title = all_titles)) 
groups = df.groupby('label')
fig, ax = plt.subplots(figsize=(17, 9))
ax.margins(0.05)
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=name, color=color[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',         
        which='both',      
        bottom='off',      
        top='off',        
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         
        which='both',      
        left='off',      
        top='off',         
        labelleft='off')
    
ax.legend(numpoints=1)
for i in range(len(df)):
    ax.text(df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['title'], size=8)  
plt.show()

In [47]:
df = pd.DataFrame(dict(x=all_xs, y=all_ys, label=all_labels, title = all_titles))
df_ = df[df['label'] == 0]
print(df_)

                  x             y  label title
0     -3.010915e-07  8.795091e-07      0     0
1     -3.010917e-07  8.795060e-07      0     0
2     -3.010905e-07  8.795095e-07      0     0
3     -3.010912e-07  8.795095e-07      0     0
4     -3.010895e-07  8.795088e-07      0     0
...             ...           ...    ...   ...
23619 -3.010902e-07  8.795064e-07      0     0
23620 -3.010897e-07  8.795089e-07      0     0
23621 -3.010903e-07  8.795068e-07      0     0
23622 -3.010895e-07  8.795102e-07      0     0
23623 -3.010896e-07  8.795093e-07      0     0

[23624 rows x 4 columns]


In [49]:

import pandas as pd
#%matplotlib inline 
for isf in range(len(meanclust_dist)):
    df = pd.DataFrame(dict(x=all_xs, y=all_ys, label=all_labels, title = all_titles))
    df = df[df['label'] == isf]
    groups = df.groupby('label')
    fig, ax = plt.subplots(figsize=(17, 9))
    ax.margins(0.05)
    for name, group in groups:
        ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
                label=name, color=color[name], 
                mec='none')
        ax.set_aspect('auto')
        ax.tick_params(\
            axis= 'x',         
            which='both',      
            bottom='off',      
            top='off',        
            labelbottom='off')
        ax.tick_params(\
            axis= 'y',         
            which='both',      
            left='off',      
            top='off',         
            labelleft='off')
        
    ax.legend(numpoints=1)
    for i in range(len(df)):
        ax.text(df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['title'], size=8)  
    plt.savefig('cluster'+ str(isf)+'.png')
    plt.close()
plt.show()

In [ ]:
def getnames_pos(clusternr, x_lower, x_higher, y_lower,y_higher, position, headerinf):
    clusterid_headerinfo = clusternr+4
    pos_clusterofinterest = pos[clusternr]
    headerinfos_clusterofinterest = headerinf[clusterid_headerinfo]
    #print(len(headerinfos_clusterofinterest))
    x_lowerbound= x_lower
    x_higherbound = x_higher
    y_lowerbound = y_lower
    y_higherbound = y_higher
    #get indixes for lookup in headerinfo for this range
    indexesofinterest = []
    count = 0
    for c in pos_clusterofinterest:
        if c[0]>x_lowerbound and c[0]<x_higherbound and c[1]>y_lowerbound and c[1]<y_higherbound:
            indexesofinterest.append(headerinfos_clusterofinterest[count])
        count += 1
    print(len(indexesofinterest))
    return list(set(indexesofinterest))


In [ ]:
g = (getnames_pos(3, -0.3, -0.2, -0.03, 0.016, pos, headerinfos_by_cluster))
for gg in g:
    print(gg)
h = getnames_pos(3, 0.25,0.3,-0.03, 0.016, pos, headerinfos_by_cluster)
for gg in h:
    print(gg)


In [ ]:
clusterid = 3
g1 = getnames_pos(clusterid, -0.3, -0.1, -0.3, 0.3, pos, headerinfos_by_cluster)
g2 = getnames_pos(clusterid, -0.06, 0.3, -0.3, 0.3, pos, headerinfos_by_cluster)
g3= getnames_pos(clusterid, -0.3, 0.3, -0.3, -0.11, pos, headerinfos_by_cluster)
g4= getnames_pos(clusterid, -0.3, 0.3, 0.1, 0.3, pos, headerinfos_by_cluster)
g=[]
g.append(g1)
g.append(g2)
g.append(g3)
g.append(g4)
gg=[item for sublist in g for item in sublist]
gg=list(set(gg))

In [ ]:
print(len(gg))
print(len([x for x in gg if x.lower().__contains__('id')]))
for l in gg:
    print(l)

In [ ]:
g14a = getnames_pos(14,0.18,0.3,-0.2,0.0, pos,headerinfos_by_cluster) #90 punkte, die berücksichtig werden
g14b = getnames_pos(14, -0.3,-0.17,0.0,0.1, pos, headerinfos_by_cluster) # 22 Punkte, die berücksichtigt werden

In [ ]:
print (len(g14a))
print(len (g14b))